# Run all 

In [1]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data, load_local_dict, unload_local_dict
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper

nr_payload_pre = len(load_local_dict())
nr_ads_pre = len(pd.read_csv('../data/bronze/jobs.csv'))

scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), NikitaScraper(), RegentScraper(), UpgradedScraper()]
scrapers = [UpgradedScraper()]

for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.return_raw_job_posts_data(response)
    old_payload_dict = load_local_dict()
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_ads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
    unload_local_dict(updated_payload_dict)
    unload_local_data(updated_bronze_data)


nr_payload_post = len(load_local_dict())
nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
print('Total number of new added jobs_payloads:', nr_payload_post-nr_payload_pre)


Upgraded > Response: 200
Upgraded > Nmr of scraped adds: 80
Upgraded > Parsing bronze data: 7
Total number of new added jobs: 7
Total number of new added jobs_payloads: 7


## Load bronze table 

In [ ]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
